# 📊 Data Analysis Report for Rice Leaf Disease Dataset

---

## 1. 📂 Dataset Overview

The dataset consists of **119 images** of rice leaves categorized into three classes:

- **Bacterial Leaf Blight**: 40 images  
- **Brown Spot**: 40 images  
- **Leaf Smut**: 39 images  

These images are collected from infected rice plants, each showing distinct symptoms. The objective is to classify these images into the correct disease categories.

---

## 2. 🧾 Dataset Characteristics

- **Small dataset size**: Only 119 images, which is limited for deep learning.  
- **Class distribution**: Nearly balanced (around 40 images per class).  
- **Image quality**: Varies in lighting, background, and leaf orientation.  
- **Image format**: RGB images with 3 color channels (important for disease spot detection).

---

## 3. ⚠️ Challenges with the Dataset

- **Limited sample size** → Higher risk of overfitting.  
- **Variability** → Differences in angles, lighting, and background make learning harder.  
- **Visual similarity** → Some diseases look quite similar, leading to possible misclassification.

---

## 4. 🛠️ Data Preprocessing & Cleaning

- **Resizing** → All images resized to a uniform size (e.g., 255x255 pixels).  
- **Normalization** → Pixel values scaled between 0 and 1.  
- **Shuffling and batching** → Applied for better training and to avoid bias.

---

## 5. 🔁 Data Augmentation

To improve model performance and reduce overfitting, augmentation techniques were used:

- Random **horizontal/vertical flips**  
- Random **rotations**  
- These simulate real-world variations and help the model generalize better.

---

## 6. 🔎 Exploratory Data Analysis (EDA)

- **Class count visualization** → Checked if data is balanced.  
- **Sample visualization** → Displayed images from each class to understand appearance.  
- **Label verification** → Ensured correct labeling of images.

---

## 7. ✅ Summary & Insights

- Dataset is small but usable for training with augmentation and preprocessing.  
- Data augmentation is **necessary** to improve learning and generalization.  
- Preprocessing brings uniformity to image inputs.  
- **Deep models or transfer learning** may be required due to visual similarity between disease classes.

---

## Importing Libraries and Mount drive to Import Images dataset

In [ ]:
!pip install tensorflow
import tensorflow as tf 
from tensorflow.keras import models,layers
import matplotlib.pyplot as plt

In [3]:
# set all the constants(hyperparameters)
BATCH_SIZE = 32
IMAGE_SIZE = 255
CHANNEL = 3
EPOCHS = 20
from google.colab import drive
drive.mount('/content/drive')

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory='/content/drive/MyDrive/PRCP-1001-RiceLeaf/Data',  # or your actual path here
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
class_names = dataset.class_names
class_names

## Label :-
#### This part shows the labels corresponding to the images in the batch. It's a 1D tensor with a shape of (32), indicating the label for each image in the batch. The labels seem to be integers, ranging from 0 to 2

In [ ]:
for image_batch,label_batch in dataset.take(1):
    print("Image Batch Shape : ", image_batch.shape)
    print("Single Image : ", image_batch[0])
    print("Label Image numpy : ", label_batch.numpy)

In [ ]:
len(class_names)

# EDA :-
## Visualize some of the images from our dataset

In [ ]:
plt.figure(figsize=(10, 10))
for image_batch, labels_batch in dataset.take(1):
    # Code to process the first batch of images and labels
    for i in range(12):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.title(class_names[labels_batch[i]])
        plt.axis("off")

## Function to Split Dataset
### Dataset should be bifurcated into 3 subsets, namely:

##### Training: Dataset to be used while training
##### Validation: Dataset to be tested against while training
##### Test: Dataset to be tested against after we trained a model

In [ ]:
total_batches = len(dataset)

train_size = max(1, int(0.7 * total_batches))
val_size = max(1, int(0.2 * total_batches))
test_size = total_batches - train_size - val_size

# Adjust again if needed
if test_size <= 0:
    test_size = 1
    train_size = total_batches - val_size - test_size

# Split dataset
train_ds = dataset.take(train_size)
val_ds = dataset.skip(train_size).take(val_size)
test_ds = dataset.skip(train_size + val_size)

# Print sizes
print(f"Train batches: {len(train_ds)}")
print(f"Validation batches: {len(val_ds)}")
print(f"Test batches: {len(test_ds)}")

##Building the Model
#### Creating a Layer for Resizing and Normalization Before we feed our images to network, we should be resizing it to the desired size. Moreover, to improve model performance, we should normalize the image pixel value (keeping them in range 0 and 1 by dividing by 256). This should happen while training as well as inference. Hence we can add that as a layer in our Sequential Model.

#### You might be thinking why do we need to resize (256,256) image to again (256,256). You are right we don't need to but this will be useful when we are done with the training and start using the model for predictions. At that time somone can supply an image that is not (256,256) and this layer will resize it

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(IMAGE_SIZE, IMAGE_SIZE), # Changed from layers.preprocessing.Resizing
  layers.Rescaling(1./255), # Changed from layers.preprocessing.Rescaling
])

# 📊 Data Augmentation Report

## 📝 Why Data Augmentation Was Used

In this project, the dataset consists of only **119 total images** across **three classes**:

- 40 images of **Bacterial Leaf Blight**
- 40 images of **Brown Spot**
- 39 images of **Leaf Smut**

This is a very small dataset for training a deep learning model. If trained without augmentation, the model would likely:

- Overfit to the small training set
- Fail to generalize on new or unseen images
- Show poor validation and test accuracy

To overcome this, **data augmentation** was applied.

In [ ]:
from tensorflow.keras import layers
import tensorflow as tf

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
])

# Applying Data Augmentation to Train Dataset

In [ ]:
for batch in val_ds.take(1):
    print("Validation batch found.")

for images, labels in val_ds.take(1):
    print(f"Validation batch shape: {images.shape}")

train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
print(f"Train batches: {len(train_ds)}")
print(f"Validation batches: {len(val_ds)}")

# Model Architecture

#### We use a CNN coupled with a Softmax activation in the output layer. We also add the initial layers for resizing, normalization and Data Augmentation.

In [ ]:
n_classes = 3

model = models.Sequential([
    layers.InputLayer(input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNEL)),
    resize_and_rescale,
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

In [ ]:
print(f"Length of train_ds: {len(train_ds)}")
print(f"Length of val_ds: {len(val_ds)}")
print(f"Length of test_ds: {len(test_ds)}")

if len(val_ds) == 0:
    print("Warning: val_ds is empty. Validation metrics will not be available.")

In [ ]:
model.summary()

## Compiling the Model
We use adam Optimizer, SparseCategoricalCrossentropy for losses, accuracy as a metric

In [ ]:
from tensorflow.keras.optimizers import Nadam

optimizer = Nadam(learning_rate=0.001)

model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',  # Use this if your labels are integers
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,  # Ensure val_ds is not None
    epochs=85,
    verbose=1,
)

In [ ]:
scores = model.evaluate(test_ds)

In [ ]:
model.save("model.h5")

## Plotting the Accuracy and Loss Curves

In [ ]:
print(history)
print(history.params)
print(history.history.keys())

In [ ]:
print("Available history keys:", history.history.keys())
if len(val_ds) == 0:
    print("⚠️ Warning: Validation dataset is empty. No validation accuracy/loss will be recorded.")
history.history['loss'][:5] # show loss for first 5 epochs

In [ ]:
acc = history.history['accuracy']
# val_acc = history.history['val_accuracy']

loss = history.history['loss']
# val_loss = history.history['val_loss']

In [ ]:
acc = history.history['accuracy']
val_acc = history.history.get('val_accuracy')  # use .get() in case it's None

loss = history.history['loss']
val_loss = history.history.get('val_loss')

epochs_range = range(len(acc))  # will automatically match the number of trained epochs

plt.figure(figsize=(12, 6))

# Accuracy Plot
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
if val_acc:
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

# Loss Plot
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
if val_loss:
    plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()

# Run prediction on a sample image

In [ ]:
model = tf.keras.models.load_model('model.h5')

In [ ]:
import numpy as np
for images_batch, labels_batch in test_ds.take(1):

    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()

    print("first image to predict")
    plt.imshow(first_image)
    print("actual label:",class_names[first_label])

    batch_prediction = model.predict(images_batch)
    print("predicted label:",class_names[np.argmax(batch_prediction[0])])

# 🧠 Inference Report

## 📝 Why Inference Was Used

After training the model on a small dataset, inference was done to check how well the model predicts new, unseen images.

## 🔍 What Inference Does

- Predicts classes for new images using the trained model  
- Provides confidence scores for each prediction  
- Helps verify model accuracy on test data

## ✅ Benefits of Inference

- Confirms model’s real-world performance  
- Shows actual vs predicted results  
- Measures prediction confidence

## Function for inference

In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))

        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i]]

        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")

        plt.axis("off")

In [ ]:
model.save("Plant_leaf_diseases_detection_model.h5")